In [ ]:
! pip3 install -qU guardrails-ai lark openai langchain_community langchain_experimental langchain-upstage sentence-transformers langchainhub langchain-chroma langchain matplotlib python-dotenv tavily-python ragas faiss-cpu tokenizers 

In [ ]:
%load_ext dotenv
%dotenv
# set UPSTAGE_API_KEY

In [ ]:
import warnings

warnings.filterwarnings("ignore")

In [ ]:
import gradio as gr

from langchain_upstage import ChatUpstage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain.schema import AIMessage, HumanMessage


llm = ChatUpstage()

from tavily import TavilyClient
import os

# tavily = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])

In [ ]:
# More general chat
chat_with_history_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{message}"),
    ]
)

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
rag_with_history_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question considering the history of the conversation. 
If you don't know the answer, just say that you don't know. 
---
CONTEXT:
{context}
        """,
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{message}"),
    ]
)



In [ ]:
chain = rag_with_history_prompt | llm | StrOutputParser()

In [ ]:
from langchain_chroma import Chroma
from langchain_upstage import UpstageEmbeddings
from langchain.docstore.document import Document


from langchain_upstage import UpstageGroundednessCheck
groundedness_check = UpstageGroundednessCheck()

vectorstore = Chroma(
    persist_directory = "C:/Users/JunMung/Desktop/mystudy/kaist/upstage/cookbook/data/taeho/chroma_db203",
    embedding_function=UpstageEmbeddings(model="solar-embedding-1-large"),
)

index_count = vectorstore._collection.count()
print(f"Vector store index count: {index_count}")

retriever = vectorstore.as_retriever(search_kwargs={"k":3})

from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever


metadata_field_info = [
    AttributeInfo(
        name="subject",
        description="The subject of the email",
        type="string",
    ),
    AttributeInfo(
        name="from",
        description="who sent the email",
        type="string",
    ),
    AttributeInfo(
        name="to",
        description="who received the email",
        type="string",
    ),
    AttributeInfo(
        name="date", description="\"YYYY-MM-DDTHH:MM:SS\"", type="string"
    ),
    AttributeInfo(
        name="uid", description="email unique id", type="string"
    ),
    AttributeInfo(
        name="cc", description="carbon copy", type="string"
    )
]


document_content_description = "The content of the email"

retriever = SelfQueryRetriever.from_llm(
    llm,
    vectorstore,
    document_content_description,
    metadata_field_info,
)




def get_retrieved_docs(query):
    results_docs = retriever.invoke(query)

    return results_docs


In [ ]:
def chat(message, history):
    results_docs = get_retrieved_docs(message)
    history_langchain_format = []
    for human, ai in history:
        history_langchain_format.append(HumanMessage(content=human))
        history_langchain_format.append(AIMessage(content=ai))

    for _ in range(5):
        response = chain.invoke({
            "message": message, 
            "context": results_docs,
            "history": history_langchain_format
        })
        gc_result = groundedness_check.invoke({"context":results_docs,"answer":response})
        print(results_docs)
        print(response)
        print("GC check result: ", gc_result)
        if gc_result.lower().startswith("grounded"):
            print("✅ Groundedness check passed")
            break
        else:
            print("❌ Groundedness check failed")
            continue

    return response

In [ ]:
with gr.Blocks() as demo:
    chatbot = gr.ChatInterface(
        chat,
        examples=[
            "How to eat healthy?",
            "Best Places in Korea",
            "How to make a chatbot?",
        ],
        title="Solar Chatbot",
        description="Upstage Solar Chatbot",
    )
    chatbot.chatbot.height = 300
    outputs=[gr.Textbox(label="Retrieved Email List")],
    

    

In [ ]:
if __name__ == "__main__":
    demo.launch()